In [84]:
import numpy as np
import pandas as pd
from definitions import *
from tqdm import tqdm
import cv2
from triangles import TRIANGLES
import random
import plotly.express as px

In [23]:
df_pav = pd.read_csv("./points_above_average_expanded.csv",index_col=0)

## Groups Visualization

In [14]:
mean_neutral_face_path = PROCESSED_PATH + "/00/mean-lms-00.csv"
neutral_mask = pd.read_csv(mean_neutral_face_path, index_col=0)

In [20]:
filtered_neutral_mask = neutral_mask[neutral_mask.index.isin(df_pav["points_above_average"].tolist())]
points = filtered_neutral_mask.values

In [16]:
colors = []
for i in range(len(df_pav["kmean_group"].unique())):
    colors.append((random.randint(0, 255),random.randint(0, 255),random.randint(0, 255)))

In [21]:
pav_groups = []

pav_list = df_pav["points_above_average"].tolist()
group_list = df_pav["kmean_group"].tolist()
for i in range(len(df_pav["kmean_group"].unique())):
    gp = []
    for j in range(len(pav_list)):
        if(group_list[j] == i):
            gp.append(pav_list[j])
    pav_groups.append(gp)


In [22]:
image = np.zeros(
            (int(final_image_size_height), int(final_image_size_width), 3), dtype='uint8')
x = (neutral_mask["x"]).tolist()
y = (neutral_mask["y"]).tolist()
lms = []
for i in range(len(x)):
    lms.append([int(x[i]), int(y[i])])
for i,lm in enumerate(lms):
    color = (255,255,255)
    if(i in pav_list):
        for j in range(len(pav_groups)):
            for k in pav_groups[j]:
                if(i == k):
                    color = colors[j]
                    image = cv2.putText(image, str(i), (lm), cv2.FONT_HERSHEY_SIMPLEX,0.4,color,1,cv2.LINE_AA)
    else:
        image = cv2.circle(image, (lm), radius=5, color=color, thickness=-1)


for i,color in enumerate(colors):
    image = cv2.putText(image, f'Grupo {i}', (50,50*i + 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, color, 2, cv2.LINE_AA)
cv2.imwrite("./kmeans_gps_expanded_visualization.png", image)

True

## Plot creation

In [88]:

df_aggr_happy = pd.DataFrame()

In [89]:
path = PATTERN_TEST_PATH_SAME_POINT
gp = [
    ["Grupo 00",[]],
    ["Grupo 01",[]],
    ["Grupo 02",[]],
    ["Grupo 03",[]],
    ["Grupo 04",[]],
    ["Grupo 05",[]],
    ["Grupo 06",[]],
    ["Grupo 07",[]],
    ["id_user",[]]
]
users = os.listdir(path)
for user in users:
    path_u = path+f'/{user}'
    df_aux = pd.read_csv(path_u+"/05-00_00-00.csv", index_col=0)
    df_aux =df_aux[df_aux["origins"].isin(df_pav["points_above_average"].tolist())]
    df_aux = df_aux.set_index('origins').join(df_pav.set_index('points_above_average'))
    distances_means = df_aux.groupby("kmean_group").mean()["distances"].tolist()
    for i, dist in enumerate(distances_means):
        gp[i][1].append(dist)
    gp[8][1].append(int(user))

In [90]:
for dt in gp:
    col, data = dt
    df_aggr_happy[col] = np.array(data)

In [91]:
tests_df = pd.read_csv(FORMS_PATH+"/total_scores_by_user.csv",index_col=0)

In [92]:
df_aggr_happy = df_aggr_happy.set_index('id_user').join(tests_df.set_index('id_user'))

In [93]:
df_aggr_happy.head()

,Grupo 00,Grupo 01,Grupo 02,Grupo 03,Grupo 04,Grupo 05,Grupo 06,Grupo 07,dbi_score,bai_score,asrsa_score,asrsb_score,oci_score,sqr_score,age
id_user,,,,,,,,,,,,,,,
1,16.777778,27.315789,6.181818,19.2,15.142857,40.047619,1.294118,2.588235,3.0,2.0,12.0,14.0,9.0,0.0,22
5,38.444444,48.578947,22.454545,48.1,38.571429,74.666667,14.352941,17.352941,17.0,18.0,21.0,12.0,25.0,13.0,20
7,32.333333,84.105263,28.272727,47.9,56.285714,80.238095,12.470588,10.941176,11.0,5.0,9.0,10.0,20.0,5.0,23
8,28.777778,57.684211,28.272727,21.6,24.714286,69.857143,5.176471,7.588235,2.0,6.0,13.0,10.0,14.0,2.0,21
9,15.111111,26.000000,15.181818,12.1,12.714286,35.333333,2.705882,7.529412,9.0,3.0,22.0,4.0,16.0,8.0,22


### 05 x 06

In [98]:
fig = px.scatter_3d(df_aggr_happy, x='Grupo 05', y='Grupo 06', z='dbi_score')
fig.write_html("fake_smile_verification/05x06.html")

### 01 x 07

In [97]:
fig = px.scatter_3d(df_aggr_happy, x='Grupo 01', y='Grupo 07', z='dbi_score')
fig.write_html("fake_smile_verification/01x07.html")